Retype of [this tutorial](https://habr.com/ru/company/mailru/blog/438560/) by Temkahap

In [11]:
import numpy as np
from sklearn.metrics import mean_squared_error as mse
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor

In [8]:
class RegressionTree:
    '''
    class RegressionTree решает задачу регрессии. Основан на рекурсивных вызовах, когда прописываются условия выхода из рекурсии
    '''
    def __init__(self, max_depth=3, n_epoch=10, min_size=8):
        self.max_depth = max_depth #максимальная глубина
        self.min_size = min_size #минимальный размер поддерева
        self.value = 0 #значение в поддереве(среднее по всем листьям)
        self.feature_idx = -1 #номер лучшего признака
        self.feature_treshold = 0#значение лучшего признака
        self.right = None# правый потомок
        self.left = None#левый потомок
        
    def fit(self, X, y):
        '''
        Процедура обучения дерева. На выходе получим обученную модель
        '''
        #инициализируем начальные значения
        self.value = y.mean()
        base_error = ((y - self.value) ** 2).sum()
        error = base_error
        flag = 0
        
        #ошибки в левом и правом поддереве
        prev_error_left = base_error
        prev_error_right = 0
        
        #если дошли до глубины 0 - выходим
        if self.max_depth <= 1:
            return
        
        dim_shape = X.shape[1]
        
        #значения в левом и правом поддереве
        left_value = 0
        right_value = 0
        
        #начинаем цикл по признакам
        for feat in range(dim_shape):
            
            #сортируем признаки
            idxs = np.argsort(X[:, feat])
            
            #количесвто сэмплов в левом и правом поддереве
            N = X.shape[0]
            N1, N2 = N, 0
            thres = 1
            
            #начинаем проходиться по значениям признака
            while thres < N - 1:
                N1 -= 1
                N2 += 1
                idx = idxs[thres]
                x = X[idx, feat]
                
                #пропускаем одинаковые признаки
                if thres < N - 1 and x == X[idxs[thres+1], feat]:
                    thres += 1
                    continue
                    
                #данные, которые получаются в результате такого сплита
                target_left = y[idxs][thres:]
                target_right = y[idxs][:thres]
                mean_left = y[idxs][thres:].mean()
                mean_right = y[idxs][:thres].mean()
                
                #на этом шаге уже нужно считать ошибку
                #генерируем предикты(среднее в потомках)
                left_shape = target_left.shape[0]
                right_shape = target_right.shape[0]
                mean_left_array = [mean_left for _ in range(left_shape)]
                mean_right_array = [mean_right for _ in range(right_shape)]
                
                #считаем ошибку слева и справа
                prev_error_left = N1 / N * mse(target_left, mean_left_array)
                prev_error_right = N2 / N * mse(target_right, mean_right_array)
                
                #если выполняются условия сплита - то обновляем
                if (prev_error_left + prev_error_right < error):
                    if (min(N1, N2) < self.min_size):
                        self.feature_idx = feat
                        self.feature_treshold = x
                        left_value = mean_left
                        right_value = mean_right
                        
                        flag = 1
                        error = prev_error_left + prev_error_right
                        
                thres += 1
                
        #если не нашли лучший сплит - выходим
        if self.feature_idx == -1:
            return
        
        #если дошли сюда - значит есть хорошее разбиение
        #можно обучать дальше 
        #инициализируем потомков- тоже деревья решений
        self.left = RegressionTree(self.max_depth - 1)
        self.left.value = left_value
        self.right = RegressionTree(self.max_depth - 1)
        self.right.value = right_value
        
        #индексы потомков
        idxs_l = (X[:, self.feature_idx] > self.feature_treshold)
        idxs_r = (X[:, self.feature_idx] <= self.feature_treshold)
        
        #обучаем
        self.left.fit(X[idxs_l, :], y[idxs_l])
        self.right.fit(X[idxs_r, :], y[idxs_r])
        
    def __predict(self, x):
        '''
        функция для предсказания. Идем в узлы - ищем потомков и смотрим в конце self.value, которое и будет ответом
        '''
        if self.feature_idx == -1:
            return self.value
        
        if x[self.feature_idx] > self.feature_treshold:
            return self.left.__predict(x)
        else:
            return self.right.__predict(x)
        
    def predict(self, X):
        '''
        Просто вызываем __predict для каждой строчки матрицы
        '''
        y = np.zeros(X.shape[0])
        
        for i in range(X.shape[0]):
            y[i] = self.__predict(X[i])
            return y        

Метод fit, как понятно из названия, обучает модель. На вход подаётся обучающая выборка и происходит процедура обучения дерева. Сортируя признаки, мы ищем наилучшее разбиение дерева с точки зрения уменьшения энтропии, в данном случае mse. Определить, что удалось найти хороший сплит, очень просто, достаточно выполнения двух условий. Мы не хотим, чтобы в разбиение попадало мало объектов (защита от переобучения), и средневзвешенная ошибка по mse должна быть меньше той ошибки, которая есть сейчас в дереве — мы ищем тот самый прирост information gain. Пройдя таким образом все признаки и все уникальные значения по ним, мы переберём все варианты и выберем наилучшее разбиение. А дальше делаем рекурсивный вызов на полученных разбиениях до тех пор, пока не выполнятся условия выхода из рекурсии.

Метод __predict, как понятно из названия, делает предикт. Получив на вход объект, он идёт по узлам полученного дерева — в каждом узле зафиксирован номер признака и значение по нему, и в зависимости от того, какое значение у поступившего метода объекта по этому признаку, мы идём либо в правого потомка, либо в левого, пока не дойдём до листа, в котором и будет ответ для данного объекта.

Метод predict делает то же самое, что и прошлый метод, только для группы объектов.

Импортируем всем известный набор данных о домах в Калифорнии. Это обычный датасет с данными и таргетом для решения задачи регрессии.

In [5]:
data = datasets.fetch_california_housing()
X = np.array(data.data)
y = np.array(data.target)

In [9]:
%%time
A = RegressionTree(max_depth=2)
A.fit(X, y)

Wall time: 3min 52s


In [12]:
%%time
model = DecisionTreeRegressor(max_depth=2)
model.fit(X, y)

Wall time: 576 ms
